# Análise exploratória de acórdão em BD e extração de dados do PDF

### Importar bibliotecas

In [1]:
import pandas as pd

import pyodbc

from pathlib import Path
import pdfplumber
import re

import unicodedata

## Análise de acórdão a partir do BD
[Configurar o ambiente de desenvolvimento para o desenvolvimento Python pyodbc](https://docs.microsoft.com/pt-br/sql/connect/python/pyodbc/step-1-configure-development-environment-for-pyodbc-python-development?view=sql-server-ver15)

In [2]:
sql_conn = pyodbc.connect('Trusted_Connection=yes', \
                          driver = '{SQL Server Native Client 11.0}', \
                          server = 'LUA', \
                          database = 'etcev2')

In [3]:
#005697/2017 - Cons. Kleber
#026455/2017 - Cons. Waltânia
query = """select p.cod_TCE, p.assunto, a.descricao, a.texto, a.cod_situacao, c.descricao, a.cod_tipo_ato, ta.descricao, ad.caminho, ad.texto_conteudo
from etce.protocolo p 
join etce.ato a on a.cod_protocolo = p.cod and a.cod_tipo_ato in (2, 3, 75)
    and a.data_finalizacao is not null --a.cod_situacao =410
join etce.tipo_ato ta on ta.cod = a.cod_tipo_ato
join comum.tipo_classificacao c on c.cod = a.cod_situacao
join etce.arquivo_digital ad on ad.cod = a.cod_arquivo_digital
where p.cod_TCE in ('026455/2017')"""

In [4]:
df = pd.read_sql(query, sql_conn)

In [5]:
df.head()

,cod_TCE,assunto,descricao,texto,cod_situacao,descricao,cod_tipo_ato,descricao,caminho,texto_conteudo
0,026455/2017,DENÚNCIA REF. IRREGULARIDADES NA ADMINISTRAÇÃO...,ACOCAM - G.WA - 712/2020,Gabinete da Conselheira Waltânia Alvarenga (...,410,Aprovado,2,ACÓRDÃO DE CÂMARA,/2020/07//714659/8/2/28072020_080209_8F7DF1D88...,Gabinete da Conselheira Waltânia Alvarenga (...


In [6]:
print(df.iloc[0].texto)

Gabinete da Conselheira  Waltânia Alvarenga  (86) 3215-3866  TC/026455/2017 – Denúncia. P. M. de Parnaíba, exercício 2017.  MISSÃO: Exercer o controle externo, mediante orientação, fiscalização e avaliação da gestão dos recursos públicos,  visando a sua efetiva aplicação em prol da sociedade.  1  ACÓRDÃO Nº 820/2020  PROCESSO: TC/026455/2017  ASSUNTO: DENÚNCIA – SUPOSTAS IRREGULARIDADES EM  LICITAÇÕES PARA EXECUÇÃO DE OBRAS  UNIDADE GESTORA: P. M. DE PARNAÍBA, EXERCÍCIO 2017  DENUNCIADO: FRANCISCO DE ASSIS DE MORAES SOUZA (PREFEITO)  RELATORA: CONS. WALTÂNIA MARIA N. DE SOUSA LEAL  ALVARENGA  PROCURADOR: PLÍNIO VALENTE RAMOS NETO  ADVOGADOS: SUÉLLEN VIEIRA SOARES – OAB/PI Nº 5.942 E HILLANA  MARTINA LOPES MOUSINHO NEIVA DOURADO – OAB/PI  Nº 6544  EMENTA: GESTOR MUNICIPAL. DENÚNCIA. MODALIDADE DE  LICITAÇÃO INADEQUADA AO OBJETO. AUSÊNCIA DE  PROJETO BÁSICO.  1. A modalidade Sistema de Registro de Preços não é adequada  para a realização de obras, pois fere o art. 1º da Lei nº  10.520/20

In [7]:
print(df.iloc[0].texto_conteudo)

Gabinete da Conselheira  Waltânia Alvarenga  (86) 3215-3866  TC/026455/2017 – Denúncia. P. M. de Parnaíba, exercício 2017.  MISSÃO: Exercer o controle externo, mediante orientação, fiscalização e avaliação da gestão dos recursos públicos,  visando a sua efetiva aplicação em prol da sociedade.  1  ACÓRDÃO Nº 820/2020  PROCESSO: TC/026455/2017  ASSUNTO: DENÚNCIA – SUPOSTAS IRREGULARIDADES EM  LICITAÇÕES PARA EXECUÇÃO DE OBRAS  UNIDADE GESTORA: P. M. DE PARNAÍBA, EXERCÍCIO 2017  DENUNCIADO: FRANCISCO DE ASSIS DE MORAES SOUZA (PREFEITO)  RELATORA: CONS. WALTÂNIA MARIA N. DE SOUSA LEAL  ALVARENGA  PROCURADOR: PLÍNIO VALENTE RAMOS NETO  ADVOGADOS: SUÉLLEN VIEIRA SOARES – OAB/PI Nº 5.942 E HILLANA  MARTINA LOPES MOUSINHO NEIVA DOURADO – OAB/PI  Nº 6544  EMENTA: GESTOR MUNICIPAL. DENÚNCIA. MODALIDADE DE  LICITAÇÃO INADEQUADA AO OBJETO. AUSÊNCIA DE  PROJETO BÁSICO.  1. A modalidade Sistema de Registro de Preços não é adequada  para a realização de obras, pois fere o art. 1º da Lei nº  10.520/20

In [8]:
df.iloc[0].caminho

'/2020/07//714659/8/2/28072020_080209_8F7DF1D88051423498B0340A48D1D3C4.pdf'

## Análise de acórdão a partir do PDF
https://github.com/jsvine/pdfplumber

In [9]:
#path = Path('G:/Meu Drive/Mestrado/acordaos/ACOPLE.pdf'')
path = Path('//alya/ftpd$/etce/' + df.iloc[0].caminho)
print(path)
print(path.exists())

\\alya\ftpd$\etce\2020\07\714659\8\2\28072020_080209_8F7DF1D88051423498B0340A48D1D3C4.pdf
True


In [10]:
with pdfplumber.open(path) as pdf:
    page = pdf.pages[0]
    text = page.extract_text()
text

'Gabinete da Conselheira\nWaltânia Alvarenga\n(86) 3215-3866\nMISSÃO: Exercer o controle externo, mediante orientação, fiscalização e avaliação da gestão dos recursos públicos, \nvisando a sua efetiva aplicação em prol da sociedade.\nACÓRDÃO Nº 820/2020\nPROCESSO:  TC/026455/2017 \nASSUNTO:  DENÚNCIA  –  SUPOSTAS  IRREGULARIDADES  EM \nLICITAÇÕES PARA EXECUÇÃO DE OBRAS\nUNIDADE GESTORA: P. M. DE PARNAÍBA, EXERCÍCIO 2017\nDENUNCIADO:      FRANCISCO DE ASSIS DE MORAES SOUZA (PREFEITO)\nRELATORA:  CONS.  WALTÂNIA  MARIA  N.  DE  SOUSA  LEAL \nALVARENGA \nPROCURADOR:  PLÍNIO VALENTE RAMOS NETO\nADVOGADOS: SUÉLLEN VIEIRA SOARES – OAB/PI Nº 5.942 E HILLANA \nMARTINA LOPES MOUSINHO NEIVA DOURADO – OAB/PI \nNº 6544\nEMENTA: GESTOR MUNICIPAL. DENÚNCIA. MODALIDADE DE \nLICITAÇÃO  INADEQUADA  AO  OBJETO.  AUSÊNCIA  DE \nPROJETO BÁSICO.\n1. A modalidade Sistema de Registro de Preços não é adequada \npara  a  realização  de  obras,  pois  fere  o  art.  1º  da  Lei  nº \n10.520/2002).\n2. A ausênci

In [11]:
with pdfplumber.open(path) as pdf:
    page = pdf.pages[0]
    text = page.extract_text()
    
    for line in text.split('\n'):
        print(line)

Gabinete da Conselheira
Waltânia Alvarenga
(86) 3215-3866
MISSÃO: Exercer o controle externo, mediante orientação, fiscalização e avaliação da gestão dos recursos públicos, 
visando a sua efetiva aplicação em prol da sociedade.
ACÓRDÃO Nº 820/2020
PROCESSO:  TC/026455/2017 
ASSUNTO:  DENÚNCIA  –  SUPOSTAS  IRREGULARIDADES  EM 
LICITAÇÕES PARA EXECUÇÃO DE OBRAS
UNIDADE GESTORA: P. M. DE PARNAÍBA, EXERCÍCIO 2017
DENUNCIADO:      FRANCISCO DE ASSIS DE MORAES SOUZA (PREFEITO)
RELATORA:  CONS.  WALTÂNIA  MARIA  N.  DE  SOUSA  LEAL 
ALVARENGA 
PROCURADOR:  PLÍNIO VALENTE RAMOS NETO
ADVOGADOS: SUÉLLEN VIEIRA SOARES – OAB/PI Nº 5.942 E HILLANA 
MARTINA LOPES MOUSINHO NEIVA DOURADO – OAB/PI 
Nº 6544
EMENTA: GESTOR MUNICIPAL. DENÚNCIA. MODALIDADE DE 
LICITAÇÃO  INADEQUADA  AO  OBJETO.  AUSÊNCIA  DE 
PROJETO BÁSICO.
1. A modalidade Sistema de Registro de Preços não é adequada 
para  a  realização  de  obras,  pois  fere  o  art.  1º  da  Lei  nº 
10.520/2002).
2. A ausência de projeto básico na c

In [12]:
with pdfplumber.open(path) as pdf:
    page = pdf.pages[0]
    text = page.extract_text(x_tolerance=2, y_tolerance=0)
    
    for line in text.split('\n'):
        print(line)

Gabinete da Conselheira
Waltânia Alvarenga
(86) 3215-3866
MISSÃO: Exercer o controle externo, mediante orientação, fiscalização e avaliação da gestão dos recursos públicos, 
visando a sua efetiva aplicação em prol da sociedade.
ACÓRDÃO Nº 820/2020
PROCESSO:
  TC/026455/2017 
ASSUNTO:
  DENÚNCIA  –  SUPOSTAS  IRREGULARIDADES  EM 
LICITAÇÕES PARA EXECUÇÃO DE OBRAS
UNIDADE GESTORA:
 P. M. DE PARNAÍBA, EXERCÍCIO 2017
DENUNCIADO:
      FRANCISCO DE ASSIS DE MORAES SOUZA (PREFEITO)
RELATORA:
  CONS.  WALTÂNIA  MARIA  N.  DE  SOUSA  LEAL 
ALVARENGA 
PROCURADOR:
  PLÍNIO VALENTE RAMOS NETO
ADVOGADOS:
SUÉLLEN VIEIRA SOARES – OAB/PI Nº 5.942 E HILLANA 
MARTINA LOPES MOUSINHO NEIVA DOURADO – OAB/PI 
Nº 6544
EMENTA: 
GESTOR MUNICIPAL. DENÚNCIA. MODALIDADE DE 
LICITAÇÃO  INADEQUADA  AO  OBJETO.  AUSÊNCIA  DE 
PROJETO BÁSICO.
1. A modalidade Sistema de Registro de Preços não é adequada 
art.  1º  da  Lei  nº 
para  a  realização  de  obras,  pois  fere  o 
10.520/2002).
2. A ausência de projeto bási

In [13]:
lines = text.split('\n')
lines

['Gabinete da Conselheira',
 'Waltânia Alvarenga',
 '(86) 3215-3866',
 'MISSÃO: Exercer o controle externo, mediante orientação, fiscalização e avaliação da gestão dos recursos públicos, ',
 'visando a sua efetiva aplicação em prol da sociedade.',
 'ACÓRDÃO Nº 820/2020',
 'PROCESSO:',
 '  TC/026455/2017 ',
 'ASSUNTO:',
 '  DENÚNCIA  –  SUPOSTAS  IRREGULARIDADES  EM ',
 'LICITAÇÕES PARA EXECUÇÃO DE OBRAS',
 'UNIDADE GESTORA:',
 ' P. M. DE PARNAÍBA, EXERCÍCIO 2017',
 'DENUNCIADO:',
 '      FRANCISCO DE ASSIS DE MORAES SOUZA (PREFEITO)',
 'RELATORA:',
 '  CONS.  WALTÂNIA  MARIA  N.  DE  SOUSA  LEAL ',
 'ALVARENGA ',
 'PROCURADOR:',
 '  PLÍNIO VALENTE RAMOS NETO',
 'ADVOGADOS:',
 'SUÉLLEN VIEIRA SOARES – OAB/PI Nº 5.942 E HILLANA ',
 'MARTINA LOPES MOUSINHO NEIVA DOURADO – OAB/PI ',
 'Nº 6544',
 'EMENTA: ',
 'GESTOR MUNICIPAL. DENÚNCIA. MODALIDADE DE ',
 'LICITAÇÃO  INADEQUADA  AO  OBJETO.  AUSÊNCIA  DE ',
 'PROJETO BÁSICO.',
 '1. A modalidade Sistema de Registro de Preços não é adequada '

In [14]:
lines[2:8]

['(86) 3215-3866',
 'MISSÃO: Exercer o controle externo, mediante orientação, fiscalização e avaliação da gestão dos recursos públicos, ',
 'visando a sua efetiva aplicação em prol da sociedade.',
 'ACÓRDÃO Nº 820/2020',
 'PROCESSO:',
 '  TC/026455/2017 ']

## Extração dos dados do acódão

In [15]:
words = page.extract_words(keep_blank_chars=True)
words

[{'x0': Decimal('376.959'),
  'x1': Decimal('519.699'),
  'top': Decimal('38.832'),
  'bottom': Decimal('55.043'),
  'text': 'Gabinete da Conselheira'},
 {'x0': Decimal('407.118'),
  'x1': Decimal('519.698'),
  'top': Decimal('56.191'),
  'bottom': Decimal('72.402'),
  'text': 'Waltânia Alvarenga'},
 {'x0': Decimal('453.230'),
  'x1': Decimal('519.700'),
  'top': Decimal('73.807'),
  'bottom': Decimal('86.277'),
  'text': '(86) 3215-3866'},
 {'x0': Decimal('103.923'),
  'x1': Decimal('521.483'),
  'top': Decimal('107.519'),
  'bottom': Decimal('117.495'),
  'text': 'MISSÃO: Exercer o controle externo, mediante orientação, fiscalização e avaliação da gestão dos recursos públicos, '},
 {'x0': Decimal('214.662'),
  'x1': Decimal('408.334'),
  'top': Decimal('118.201'),
  'bottom': Decimal('128.177'),
  'text': 'visando a sua efetiva aplicação em prol da sociedade.'},
 {'x0': Decimal('238.867'),
  'x1': Decimal('370.627'),
  'top': Decimal('137.890'),
  'bottom': Decimal('155.074'),
  'tex

In [16]:
def extrac_lines(words):
    bottom_line = 0
    lines = []
    for word in words:
        actual_bottom = int(word['bottom'])
        if (actual_bottom - bottom_line) > 5:  # Nova linha
            if (bottom_line != 0): # se não for primeira linha, adiciona
                lines.append({'x0': initial_x0, 'text': line_text})

            line_text = ''
            bottom_line = actual_bottom
            initial_x0 = int(word['x0'])

        line_text = line_text + word['text']
        #x0, x1, top, bottom, text = word
    
    return lines

In [17]:
lines = extrac_lines(words)
lines

[{'x0': 376, 'text': 'Gabinete da Conselheira'},
 {'x0': 407, 'text': 'Waltânia Alvarenga'},
 {'x0': 453, 'text': '(86) 3215-3866'},
 {'x0': 103,
  'text': 'MISSÃO: Exercer o controle externo, mediante orientação, fiscalização e avaliação da gestão dos recursos públicos, '},
 {'x0': 214, 'text': 'visando a sua efetiva aplicação em prol da sociedade.'},
 {'x0': 238, 'text': 'ACÓRDÃO Nº 820/2020'},
 {'x0': 85, 'text': 'PROCESSO: TC/026455/2017 '},
 {'x0': 85, 'text': 'ASSUNTO: DENÚNCIA – SUPOSTAS IRREGULARIDADES EM '},
 {'x0': 205, 'text': 'LICITAÇÕES PARA EXECUÇÃO DE OBRAS'},
 {'x0': 85, 'text': 'UNIDADE GESTORA: P. M. DE PARNAÍBA, EXERCÍCIO 2017'},
 {'x0': 85,
  'text': 'DENUNCIADO:     FRANCISCO DE ASSIS DE MORAES SOUZA (PREFEITO)'},
 {'x0': 85, 'text': 'RELATORA: CONS. WALTÂNIA MARIA N. DE SOUSA LEAL '},
 {'x0': 205, 'text': 'ALVARENGA '},
 {'x0': 85, 'text': 'PROCURADOR: PLÍNIO VALENTE RAMOS NETO'},
 {'x0': 85,
  'text': 'ADVOGADOS:SUÉLLEN VIEIRA SOARES – OAB/PI Nº 5.942 E HILLANA '

In [18]:
# Função para tirar acentos, remover espaços duplicados
# Fonte: https://rogeriopradoj.com/2019/07/14/como-tirar-acentos-de-string-no-python-transliterate-unicodedata-e-unidecode//
def clean_text(text):
    return unicodedata.normalize("NFD", text).encode('ASCII','ignore').decode('ASCII').replace("  ", " ").casefold().strip()

print(clean_text('ACÓRDÃO Nº 1.128/17 '))

acordao n 1.128/17


In [19]:
print(clean_text(' ').isspace())

False


In [20]:
page.width

Decimal('595.350')

In [21]:
def extrai_dados_acordao(path):
        
    def muda_parte():
        return partes_acordao.pop(0), ''
    
    def eh_vazio(string):
        return not string.strip()
    
    def eh_bloco_ementa(x0):
        #return int(x0) > (int(page.width)/2)  # Deveria ser assim (depois da metade, mas 026455/2017 (Cons. Waltânia) não está)
        return int(x0) > 210  
    
    partes_acordao = ['Acordao', 'info_processuais', 'ementa', 'sumario', 'disposicoes']
    dados = []
    cabecalho = []

    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            words = page.extract_words(keep_blank_chars=True)
            lines = extrac_lines(words)

            if page.page_number == 1:
                parte, texto_parte = muda_parte()
                testar_se_cabecalho = False
            else:
                testar_se_cabecalho = True

            for line in lines:
                text = clean_text(line['text'])
                if testar_se_cabecalho:
                    if text in cabecalho: 
                        print('cabeçalho: ', text)
                        continue
                    else: testar_se_cabecalho = False
                        
                if parte == 'Acordao':
                    if text.startswith('acordao'):
                        dados.append({f'{parte}': line['text'].strip()})
                        parte, texto_parte = muda_parte()
                    else:
                        cabecalho.append(text)
                    
                    continue
                elif parte == 'info_processuais':
                    if eh_vazio(text): continue
                    if not eh_bloco_ementa(line['x0']):
                        texto_parte = texto_parte + line['text'].strip() + '\n'
                    else:
                        dados.append({f'{parte}': texto_parte.strip()})
                        parte, texto_parte = muda_parte()
                        texto_parte = texto_parte + line['text'].strip() + '\n'
                        continue
                elif parte == 'ementa':
                    if eh_bloco_ementa(line['x0']) and not text.startswith('sumario'):
                        texto_parte = texto_parte + line['text'].strip() + '\n'
                    else:
                        dados.append({f'{parte}': texto_parte.strip()})
                        parte, texto_parte = muda_parte()
                        texto_parte = texto_parte + line['text'].strip() + '\n'
                        continue
                elif parte == 'sumario':
                    if eh_bloco_ementa(line['x0']):
                        texto_parte = texto_parte + line['text'].strip() + '\n'
                    else:
                        dados.append({f'{parte}': texto_parte.strip()})
                        parte, texto_parte = muda_parte()
                        texto_parte = texto_parte + line['text'].strip() + '\n'
                        continue
                elif parte == 'disposicoes':
                    texto_parte = texto_parte + line['text'].strip() + '\n'

        dados.append({f'{parte}': texto_parte.strip()})
    
    return dados


In [22]:
path = Path('//alya/ftpd$/etce/' + df.iloc[0].caminho)
extrai_dados_acordao(path)

cabeçalho:  gabinete da conselheira
cabeçalho:  waltania alvarenga
cabeçalho:  (86) 3215-3866
cabeçalho:  missao: exercer o controle externo, mediante orientacao, fiscalizacao e avaliacao da gestao dos recursos publicos,
cabeçalho:  visando a sua efetiva aplicacao em prol da sociedade.


[{'Acordao': 'ACÓRDÃO Nº 820/2020'},
 {'info_processuais': 'PROCESSO: TC/026455/2017\nASSUNTO: DENÚNCIA – SUPOSTAS IRREGULARIDADES EM\nLICITAÇÕES PARA EXECUÇÃO DE OBRAS\nUNIDADE GESTORA: P. M. DE PARNAÍBA, EXERCÍCIO 2017\nDENUNCIADO:     FRANCISCO DE ASSIS DE MORAES SOUZA (PREFEITO)\nRELATORA: CONS. WALTÂNIA MARIA N. DE SOUSA LEAL\nALVARENGA\nPROCURADOR: PLÍNIO VALENTE RAMOS NETO\nADVOGADOS:SUÉLLEN VIEIRA SOARES – OAB/PI Nº 5.942 E HILLANA\nMARTINA LOPES MOUSINHO NEIVA DOURADO – OAB/PI\nNº 6544'},
 {'ementa': 'EMENTA: GESTOR MUNICIPAL. DENÚNCIA. MODALIDADE DE\nLICITAÇÃO INADEQUADA AO OBJETO. AUSÊNCIA DE\nPROJETO BÁSICO.\n1. A modalidade Sistema de Registro de Preços não é adequada\npara a realização de obras, pois fere o art. 1º da Lei nº\n10.520/2002).\n2. A ausência de projeto básico na composição de procedimentos\nlicitatórios para execução de obras afronta o art. 6º, IX c/c art.\n7º, § 2º, I e art. 40, § 2º, I da lei nº 8.666/93.'},
 {'sumario': 'SUMÁRIO: Denúncia – Prefeitura Muni